In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '....'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from get_data import get_data
import pytz
from datetime import datetime
import talib as ta
import random
# from backbone.trader_bot import TraderBot
random.seed(42)


In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [4]:

tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Stocks_group' in symbol.path and symbol.spread < 15]
print(tickers)


# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2021, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    # mt5.TIMEFRAME_M1,
    # mt5.TIMEFRAME_M2,
    # mt5.TIMEFRAME_D1,
    # mt5.TIMEFRAME_M3,
    # mt5.TIMEFRAME_M4,
    # mt5.TIMEFRAME_M5,
    mt5.TIMEFRAME_M15,
    mt5.TIMEFRAME_M30,
    mt5.TIMEFRAME_H1,
    mt5.TIMEFRAME_H4,
]

symbols = get_data(tickers, intervals, date_from, date_to)

['AAPLm', 'ABBVm', 'ABTm', 'ADBEm', 'ADPm', 'AMDm', 'AMGNm', 'AMTm', 'AMZNm', 'ATVIm', 'AVGOm', 'BAm', 'BABAm', 'BACm', 'BIIBm', 'BMYm', 'Cm', 'CHTRm', 'CMCSAm', 'CMEm', 'COSTm', 'CSCOm', 'CSXm', 'CVSm', 'EAm', 'EBAYm', 'EQIXm', 'Fm', 'FBm', 'GILDm', 'GOOGLm', 'HDm', 'IBMm', 'INTCm', 'ISRGm', 'JNJm', 'JPMm', 'KOm', 'LINm', 'LLYm', 'LMTm', 'MAm', 'MCDm', 'MDLZm', 'MMMm', 'MOm', 'MRKm', 'MSm', 'NFLXm', 'NKEm', 'ORCLm', 'PEPm', 'PFEm', 'PGm', 'PMm', 'PYPLm', 'REGNm', 'SBUXm', 'Tm', 'TMOm', 'TMUSm', 'TSLAm', 'UNHm', 'UPSm', 'Vm', 'VRTXm', 'VZm', 'WFCm', 'WMTm', 'XOMm', 'AMCm', 'BBm', 'BBBYm', 'BEKEm', 'BIDUm', 'BILIm', 'BRQSm', 'BYNDm', 'CANm', 'EDUm', 'FTNTm', 'FUTUm', 'IQm', 'JDm', 'LIm', 'NIOm', 'NTESm', 'PDDm', 'RLXm', 'TALm', 'TIGRm', 'TMEm', 'TSMm', 'VIPSm', 'XPEVm', 'YUMCm', 'ZTOm', 'METAm']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AAPLm
AAPLm
AAPLm
AAPLm
ABBVm
ABBVm
ABBVm
ABBVm
ABTm
ABTm
ABTm
ABTm
ADBEm
ADBEm
ADBEm
ADBEm
ADPm
ADPm
ADPm
ADP

In [5]:
max_start_date = None
intervals_start_dates = {}

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates



{15: Timestamp('2021-10-27 17:15:00'),
 30: Timestamp('2021-10-27 17:15:00'),
 16385: Timestamp('2021-10-27 17:15:00'),
 16388: Timestamp('2021-10-27 17:15:00')}

In [6]:
from pandas import Timestamp
limit_date_train = Timestamp('2022-10-01 00:00:00')
limit_date_train

Timestamp('2022-10-01 00:00:00')

In [7]:
def diff_pips(price1, price2, pip_value, absolute=True):
    if absolute:
        difference = abs(price1 - price2)
    else:
        difference = price1 - price2
    pips = difference / pip_value
    
    return pips


class ShortMomentum(Strategy):
    risk = 1
    atr_multiplier = 1.5
    risk_reward = 0.5
    pip_value = 0.01
    bars_in_position = 0
    max_hold_period = 5
    
    def init(self):
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)
        self.adx = self.I(ta.ADX, self.data.High, self.data.Low, self.data.Close )
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=100)
        
    def next(self):
        
        if self.position:
            self.bars_in_position += 1
            
            if self.bars_in_position >= self.max_hold_period:
                self.position.close()
                self.bars_in_position = 0
        
        else:
            if self.adx[-1] < 25:
                return
            
            go_long = self.data.Close[-1] > self.sma[-1]
            
            if go_long and self.data.Close[-1] > self.data.Close[-2]:
                
                sl = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                tp = self.data.Close[-1] + self.atr_multiplier * self.atr[-1] * self.risk_reward
                
                sl_pips = diff_pips(self.data.Close[-1], sl, pip_value=self.pip_value, absolute=True)
                account_currency_risk = self.equity * (self.risk / 100)
                units = round(account_currency_risk / (self.pip_value * sl_pips))
                
                self.buy(
                    size=units, 
                    sl=sl,
                    tp=tp
                )
            
            if not go_long and self.data.Close[-1] < self.data.Close[-2]:
                
                sl = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                tp = self.data.Close[-1] - self.atr_multiplier * self.atr[-1] * self.risk_reward
                
                sl_pips = diff_pips(self.data.Close[-1], sl, pip_value=self.pip_value, absolute=True)
                account_currency_risk = self.equity * (self.risk / 100)
                units = round(account_currency_risk / (self.pip_value * sl_pips))
                
                self.sell(
                    size=units, 
                    sl=sl,
                    tp=tp
                )

In [8]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression


strategies = [
    ShortMomentum, 
]

experiments = parameter_combinations = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:

    start_date = intervals_start_dates[interval]

    bt_train = Backtest(
        symbols[ticker][interval].loc[start_date:limit_date_train], 
        strategy,
        commission=7e-4,
        cash=15_000, 
        margin=1/30
    )
    
    stats = bt_train.run()
    
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)

    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],
        'stability_ratio':[stability_ratio],
        'return':[stats['Return [%]']],
        'final_eq':[stats['Equity Final [$]']],
        'drawdown':[stats['Max. Drawdown [%]']],
        'drawdown_duration':[stats['Max. Drawdown Duration']],
        'win_rate':[stats['Win Rate [%]']], 
        'sharpe_ratio':[stats['Sharpe Ratio']],
        'trades':[stats['# Trades']],
        'avg_trade_percent':[stats['Avg. Trade [%]']],
        'exposure':[stats['Exposure Time [%]']],
        'final_equity':[stats['Equity Final [$]']],
        'Duration':[stats['Duration']],

    })

    performance = pd.concat([performance, df_stats])

performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)
# performance.drawdown_duration = performance.drawdown_duration.dt.days



In [9]:

filter_performance = performance[
    (performance['return']>0) 
    # & (performance['return/dd']>=2) 
    # & (performance['stability_ratio'] > 0.8)
    # & (performance['interval'] <= 16385)
].sort_values(by=['custom_metric'], ascending=[False]).drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,ShortMomentum,EDUm,30,0.668318,21.374284,18206.142618,11.556255,141 days 06:00:00,58.097166,0.814535,494,-0.003467,43.938879,18206.142618,338 days 02:00:00,1.849586,10.561905
0,ShortMomentum,Fm,16388,0.832232,11.477238,16721.585662,3.592293,174 days 04:00:00,63.076923,1.876906,65,0.633824,28.284672,16721.585662,337 days 04:00:00,3.194962,10.470949
0,ShortMomentum,ADPm,16388,0.795711,7.952996,16192.949327,3.312743,120 days 04:00:00,57.142857,1.085040,70,0.211282,31.021898,16192.949327,337 days 04:00:00,2.400728,7.860675
0,ShortMomentum,NFLXm,16388,0.699090,6.874272,16031.140861,3.484349,141 days 08:00:00,65.656566,1.091101,99,0.269719,31.923077,16031.140861,337 days 08:00:00,1.972900,7.059485
0,ShortMomentum,Cm,16385,0.594753,10.363723,16554.558500,8.885693,141 days 01:00:00,60.128617,0.853506,311,0.044918,35.179576,16554.558500,337 days 08:00:00,1.166338,6.020711
0,ShortMomentum,AMCm,16385,0.754224,7.970268,16195.540270,7.039003,108 days 03:00:00,58.855586,0.661748,367,0.020905,42.057684,16195.540270,337 days 09:00:00,1.132301,5.857568
0,ShortMomentum,JPMm,16385,0.829309,9.859263,16478.889502,9.254174,102 days 04:00:00,59.227468,0.948372,233,0.073713,39.569231,16478.889502,337 days 06:00:00,1.065386,5.245225
0,ShortMomentum,XPEVm,16388,0.463672,6.806133,16020.919987,5.049560,175 days 08:00:00,60.194175,1.062162,103,0.215555,29.987130,16020.919987,337 days 08:00:00,1.347866,5.225230
0,ShortMomentum,UPSm,16388,0.100710,5.221241,15783.186104,3.539906,157 days 00:00:00,59.701493,1.078160,67,0.166551,29.927007,15783.186104,337 days 04:00:00,1.474966,4.852758
0,ShortMomentum,ORCLm,16388,0.354944,4.632987,15694.947980,3.206009,210 days 00:00:00,62.686567,1.010739,67,0.170066,30.474453,15694.947980,337 days 04:00:00,1.445095,4.647855


In [16]:
# for ticker, interval in zip(portfolio, intervals):
bt_train = Backtest(
    symbols['EDUm'][30].loc[start_date:limit_date_train], 
    ShortMomentum,
    commission=7e-4,
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()

bt_train.plot(filename=f'./{ticker}.html', resample=False)

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p6799', ...)

In [11]:
from utils import walk_forward
from utils import plot_full_equity_curve
from utils import get_wfo_stats

def  optim_func(series):
    return (series['Return [%]'] /  (1 + (-1*series['Max. Drawdown [%]']))) * np.log(1 + series['# Trades'])

def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    return stability_ratio


for ticker, interval in zip(portfolio, intervals):
    
    print(ticker, interval)
    
    lookback_bars = 1000
    validation_bars = 250
    warmup_bars = 400

    params = {
        'cum_rsi_open_threshold' : [55, 65, 75, 85, 90],
        'cum_rsi_close_threshold' : [55, 65, 75, 85, 90],
        'maximize': optim_func_2
    }

    stats = walk_forward(
        Macd,
        symbols[ticker][interval], 
        lookback_bars=lookback_bars,
        validation_bars=validation_bars,
        warmup_bars=warmup_bars, 
        params=params,
        commission=7e-4, 
        margin=1/30, 
        cash=15_000
    )

    plot_full_equity_curve(
        symbols[ticker][interval], 
        stats, 
        warmup_bars=warmup_bars,
        lookback_bars=lookback_bars, 
        overlay_price=True
    )

    wfo_stats = get_wfo_stats(stats, warmup_bars, symbols[ticker][interval])

    for k, v in wfo_stats.items():
        print(k, v)

EDUm 30


NameError: name 'Macd' is not defined